In [ ]:
#from __future__ import absolute_import
#from __future__ import division
#from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

def cnn_model_fn(features, labels, mode):
    input_layer = tf.reshape(features["x"], [-1, 256, 1])

  # Convolutional Layer and Pooling Layer #1
    conv1 = tf.layers.conv1d(
      inputs = input_layer,
      filters = 3,
      kernel_size = 39,
      padding = "same",
      activation = tf.nn.relu)
    pool1 = tf.layers.max_pooling1d(inputs=conv1, pool_size= 2, strides=2)

  # Convolutional Layer and Pooling Layer #2
    conv2 = tf.layers.conv1d(
      inputs = pool1,
      filters = 10,
      kernel_size = 9,
      padding = "same",
      activation = tf.nn.relu)
    pool2 = tf.layers.max_pooling1d(inputs = conv2, pool_size = 2, strides = 2)

  # Convolutional Layer and Pooling Layer #3
    conv3 = tf.layers.conv1d(
      inputs = pool2,
      filters = 10,
      kernel_size = 4,
      padding = "same",
      activation = tf.nn.relu)
    pool3 = tf.layers.max_pooling1d(inputs = conv3, pool_size = 2, strides = 2)

  # Convolutional Layer and Pooling Layer #4
    conv4 = tf.layers.conv1d(
      inputs = pool3,
      filters = 10,
      kernel_size = 3,
      padding = "same",
      activation = tf.nn.relu)
    pool4 = tf.layers.max_pooling1d(inputs = conv4, pool_size = 2, strides = 2)

  # Fully Connected Layer
    pool4_flat = tf.reshape(pool4, [-1, 16 * 10])
    dense = tf.layers.dense(inputs = pool4_flat, units = 256, activation = tf.nn.relu)
    dropout = tf.layers.dropout(
    inputs = dense, rate = 0.3, training = mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=23)

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name = "softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
    #class_weights = tf.constant([[1.0, 2.0, 3.0]])
  # deduce weights for batch samples based on their true label
    #weights = tf.reduce_sum(class_weights * labels, axis=1)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
        labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


if __name__ == "__main__":
    tf.app.run()

In [30]:
from collections import Counter
import wfdb
import matplotlib.pyplot as plt
import numpy as np
import math
import tensorflow as tf

#Heart rate calculation and record plotting
def peaks(x, peak_indices, fs):
    hrs = wfdb.processing.compute_hr(x.shape[0], peak_indices, fs);
    return hrs;

#Single beats extraction for record
def extract_beats(record, R_indices, peaks_hr, freq):
    
    flat_record = [item for sublist in record for item in sublist];
    ecg_beats = [];
    
    #Extract beats of different lengths depending on heart rate
    for i in range(0, len(R_indices)):
        hr = peaks_hr[R_indices[i]].item();
        if(math.isnan(hr)):
            hr = 70.0;
        samples_per_beat = int(freq*(60.0/hr))
        start = R_indices[i]-int(samples_per_beat/2);
        if(start<0):
            start = 0;
        end = start + samples_per_beat;
        ecg_beats.append(np.array(flat_record[start:end]));
    
    #Resample beats to get fixed input size for classification
    ecg_nparray = np.empty((len(ecg_beats), 256));
    for i in range(0,len(ecg_beats)):    
        ecg_beats[i], _ = wfdb.processing.resample_sig(x=ecg_beats[i],fs=len(ecg_beats[i]), fs_target=256);
        ecg_nparray[i] = ecg_beats[i];
            
    return ecg_nparray;

def read_record(rec):
    # Load the wfdb record and the physical samples
    record = wfdb.rdrecord('./dataset/'+rec, channels=[0], return_res = 32);
    annotation = wfdb.rdann('./dataset/'+rec, 'atr', summarize_labels=True);
    freq = record.fs;
    sig = record.p_signal;
    sig = wfdb.processing.normalize_bound(sig, lb=0, ub=1);

    for idx, val in enumerate(sig):    
        record.p_signal[idx,0] = val;

    peak_indices = annotation.sample;
    peaks_hr = peaks(x=record.p_signal, peak_indices=peak_indices, fs=record.fs);
    ecg_beats = extract_beats(record.p_signal, peak_indices, peaks_hr,freq);
    
    return ecg_beats, annotation;
    
beats = [];
annotations = [];
records = [100,101,102,103,104,105,106,107,108,109,111,112,113,114,115,116,117,118,119,121,122,123,124,
          200,201,202,203,205,207,208,209,210,212,213,214,215,217,219,220,221,222,223,228,230,231,232,233,234]

for i, rec in enumerate(records):
    rec_beats , annotation = read_record(str(rec));
    beats.append(rec_beats);
    annotations.append(annotation.symbol);
    ###############     Saving input in text files     ###############
    #file = open("./Text_records/Normalized/Rec_"+str(rec)+".txt", "w");
    #for j in range(len(beats[i])):
    #    file.write(str(beats[i][j]) + "\t");
    #    file.write(annotations[i][j] + "\n");
    #file.close();
    
flat_beats = [item for sublist in beats for item in sublist]
flat_ann = [item for sublist in annotations for item in sublist]
np_flat_beats = np.asarray(flat_beats, dtype = np.float32);
beats = [];
annotations = [];

Ann_Dict = {
    'N': 0,
    'L': 1,
    'R': 2,
    'V': 3,
    '/': 4,
    'A': 5,
    '+': 6,
    'f': 7,
    'F': 8,
    '~': 9,
    '!': 10,
    '"': 11,
    'j': 12,
    'x': 13,
    'a': 14,
    '|': 15,
    'E': 16,
    'J': 17,
    'Q': 18,
    'e': 19,
    ']': 20,
    '[': 21,
    'S': 22
};

labels = [];
for i in flat_ann:
    labels.append(Ann_Dict[i]);

np_labels = np.asarray(labels, dtype = np.int32);

Ann_Counter = Counter(flat_ann);
total_beats_count = len(flat_beats);

weights = [];
for index, value in enumerate(Ann_Counter.values()):
    weights.append(value/total_beats_count);
    
weights.sort(reverse = True);
print("Total no. of beats = " + str(total_beats_count));
print("Weights: " + str(weights));
print("Sum of weights: " + str(sum(weights)));
#print(Ann_Counter);
print(labels);


classifier_ = tf.estimator.Estimator(model_fn = cnn_model_fn, model_dir = "/tmp/mnist_convnet_model")

tensors_to_log = {"probabilities": "softmax_tensor"};
logging_hook = tf.train.LoggingTensorHook(tensors = tensors_to_log, every_n_iter = 500);

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {"x": np_flat_beats},
    y = np_labels,
    batch_size = 200,
    num_epochs = None,
    shuffle = True);

classifier_.train(
    input_fn = train_input_fn,
    steps = 20000,
    hooks = [logging_hook]);

eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {"x": np_flat_beats},
    y = np_labels,
    num_epochs = 1,
    shuffle = False);

eval_results = classifier_.evaluate(input_fn = eval_input_fn);
print(eval_results);

'''
for i in range(0,N):
    Y[i] = AnnDict[annotations[0][i]];
'''


Total no. of beats = 112647
Weights: [0.6662583113620425, 0.07168411054000551, 0.0644402425275418, 0.06329507221674789, 0.06238958871518993, 0.022601578382025266, 0.01146058039716992, 0.008717498024803146, 0.007128463252461229, 0.0054684101662716275, 0.004190080517013325, 0.0038793753939297096, 0.0020328992338899394, 0.0017133168215753638, 0.0013315933846440651, 0.0011718021784867772, 0.000940992658481806, 0.000736815006169716, 0.0002929505446216943, 0.00014203662769536694, 5.32637353857626e-05, 5.32637353857626e-05, 1.7754578461920867e-05]
Sum of weights: 0.9999999999999999
[6, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001D4B04BE0B8>, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_service': None, '_session_config': None, '_master': '', '_is_chief': True, '_task_id': 0, '_evaluation_master': '', '_model_dir': '/tmp/mnist_convnet_model', '_save_checkpoints_secs': 600, '_save_summary_steps': 100, '_log_step_count_steps': 100, '_global_id_in_cluster': 0, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_save_checkpoints_steps': None, '_tf_random_seed': None}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/mnist_convnet_model\model.ckpt.
INFO:tensorflow:probabilities = [[0.04352278 0.04326443 0.04427061 ... 0.

INFO:tensorflow:loss = 0.8171237, step = 1201 (1.964 sec)
INFO:tensorflow:global_step/sec: 51.5538
INFO:tensorflow:probabilities = [[0.9141221  0.0135243  0.00926788 ... 0.00151681 0.00044352 0.00071152]
 [0.8265565  0.01671544 0.03661259 ... 0.00119568 0.00112064 0.00063391]
 [0.87688965 0.01459846 0.02313706 ... 0.00199141 0.00135716 0.00072449]
 ...
 [0.7928042  0.01851633 0.0480946  ... 0.00248063 0.00224235 0.00088372]
 [0.86999387 0.01832606 0.03110755 ... 0.00493881 0.00141638 0.00133135]
 [0.8187862  0.01458071 0.05609759 ... 0.00171904 0.00138325 0.00075447]] (1.940 sec)
INFO:tensorflow:loss = 0.2220513, step = 1301 (1.941 sec)
INFO:tensorflow:global_step/sec: 51.3064
INFO:tensorflow:probabilities = [[0.26312363 0.22989975 0.1265126  ... 0.01264414 0.00390019 0.00480469]
 [0.4063     0.07929265 0.06674358 ... 0.00518917 0.00532486 0.00444684]
 [0.30749145 0.16506612 0.0634096  ... 0.00470695 0.00274249 0.00231526]
 ...
 [0.34889975 0.18425012 0.06164637 ... 0.00395299 0.003160

INFO:tensorflow:loss = 3.0340993, step = 2601 (1.965 sec)
INFO:tensorflow:global_step/sec: 52.5354
INFO:tensorflow:probabilities = [[0.8683875  0.02302213 0.0299753  ... 0.00056431 0.00072071 0.00068169]
 [0.8530242  0.02644205 0.03390782 ... 0.00061716 0.00049556 0.00031964]
 [0.82944995 0.02427229 0.04701876 ... 0.0008119  0.00023598 0.00059001]
 ...
 [0.84077907 0.02051933 0.05381463 ... 0.00074794 0.00051239 0.00038441]
 [0.8089193  0.03582875 0.02980332 ... 0.00128131 0.00072358 0.00057779]
 [0.66833395 0.06110151 0.07586131 ... 0.00137202 0.00206195 0.00071276]] (1.899 sec)
INFO:tensorflow:loss = 0.8336703, step = 2701 (1.912 sec)
INFO:tensorflow:global_step/sec: 51.078
INFO:tensorflow:probabilities = [[0.8101882  0.03958372 0.05834721 ... 0.00106416 0.00128538 0.0008644 ]
 [0.72224206 0.06486734 0.04383446 ... 0.00367227 0.00349359 0.00198076]
 [0.63188237 0.05224526 0.11853163 ... 0.00260629 0.00092442 0.00153373]
 ...
 [0.59575677 0.03448027 0.13269192 ... 0.00152806 0.0042753

INFO:tensorflow:loss = 0.68804055, step = 4001 (1.903 sec)
INFO:tensorflow:global_step/sec: 52.9257
INFO:tensorflow:probabilities = [[0.7771335  0.04220349 0.03695417 ... 0.00110719 0.00064322 0.00048632]
 [0.51215404 0.10011995 0.1136876  ... 0.00410964 0.0030321  0.00322605]
 [0.71027744 0.0310824  0.06501712 ... 0.00092774 0.00068633 0.0005888 ]
 ...
 [0.62782025 0.06382149 0.12341943 ... 0.00242697 0.00091732 0.0004147 ]
 [0.7739647  0.03193005 0.05382502 ... 0.00094339 0.00103277 0.00079097]
 [0.76138824 0.0231093  0.0572507  ... 0.00060141 0.00058857 0.00040296]] (1.903 sec)
INFO:tensorflow:loss = 0.37846622, step = 4101 (1.905 sec)
INFO:tensorflow:global_step/sec: 52.2804
INFO:tensorflow:probabilities = [[0.61563474 0.03594311 0.05535986 ... 0.00172563 0.00124367 0.00169469]
 [0.52243954 0.04220272 0.06273504 ... 0.00295189 0.00213818 0.00087919]
 [0.5538802  0.05847849 0.03002965 ... 0.00387665 0.00625313 0.00336603]
 ...
 [0.4784181  0.0596578  0.07864413 ... 0.00339731 0.0017

...

INFO:tensorflow:loss = 2.1812568, step = 15201 (1.934 sec)
INFO:tensorflow:global_step/sec: 51.7029
INFO:tensorflow:probabilities = [[0.6847511  0.06751857 0.05514936 ... 0.00044106 0.00099485 0.00047858]
 [0.69299155 0.0529617  0.05006711 ... 0.0009157  0.00072861 0.00070179]
 [0.73075193 0.05290443 0.04627414 ... 0.00038739 0.00024172 0.00018416]
 ...
 [0.65996104 0.07813077 0.06066335 ... 0.00132865 0.00044451 0.00059499]
 [0.7743853  0.04871563 0.0279354  ... 0.0003287  0.00057249 0.00040762]
 [0.6927493  0.05528447 0.04304373 ... 0.00068838 0.00049982 0.00030646]] (1.934 sec)
INFO:tensorflow:loss = 0.88889146, step = 15301 (1.936 sec)
INFO:tensorflow:global_step/sec: 51.0189
INFO:tensorflow:probabilities = [[0.62874997 0.04218018 0.06223861 ... 0.0015211  0.00079028 0.00155054]
 [0.70301676 0.04930355 0.0605558  ... 0.00064226 0.00129594 0.00031021]
 [0.6733888  0.04338516 0.06629507 ... 0.0016677  0.00103836 0.00084365]
 ...
 [0.7178733  0.03758587 0.05640402 ... 0.00068672 0.000

INFO:tensorflow:loss = 2.4255896, step = 16601 (1.927 sec)
INFO:tensorflow:global_step/sec: 51.7399
INFO:tensorflow:probabilities = [[0.74714416 0.06120699 0.0725566  ... 0.00026828 0.00019428 0.00027307]
 [0.7494973  0.06676337 0.04601278 ... 0.00031323 0.00015046 0.00024341]
 [0.74553716 0.07149175 0.07494261 ... 0.00004782 0.00020268 0.00002376]
 ...
 [0.7286754  0.05082473 0.0818086  ... 0.00008824 0.00004153 0.00010446]
 [0.7510114  0.06470557 0.08426317 ... 0.00014981 0.00010737 0.00005485]
 [0.70015705 0.07572228 0.04448392 ... 0.00019917 0.00018473 0.00007946]] (1.936 sec)
INFO:tensorflow:loss = 0.53598136, step = 16701 (1.923 sec)
INFO:tensorflow:global_step/sec: 52.6524
INFO:tensorflow:probabilities = [[0.8190495  0.04146439 0.0567135  ... 0.00006307 0.0000638  0.00006242]
 [0.6528543  0.05058048 0.0918178  ... 0.00024633 0.00010515 0.00012948]
 [0.83269745 0.03234202 0.06953421 ... 0.00002713 0.00002932 0.00003561]
 ...
 [0.81931466 0.03546157 0.0533324  ... 0.00006656 0.000

INFO:tensorflow:loss = 1.0411723, step = 18001 (1.895 sec)
INFO:tensorflow:global_step/sec: 51.1445
INFO:tensorflow:probabilities = [[0.71894467 0.10060303 0.03859868 ... 0.00009156 0.0001178  0.00006837]
 [0.77443725 0.06036324 0.03781251 ... 0.0001221  0.00019108 0.0000894 ]
 [0.78494453 0.03883014 0.04246182 ... 0.00032523 0.00023896 0.00031891]
 ...
 [0.7277382  0.05690879 0.05790146 ... 0.0003204  0.0002513  0.00017659]
 [0.6882255  0.05720304 0.02966689 ... 0.00111242 0.0008087  0.0007444 ]
 [0.50642014 0.07828812 0.06376757 ... 0.01140037 0.01029506 0.00703139]] (1.955 sec)
INFO:tensorflow:loss = 1.1512493, step = 18101 (1.951 sec)
INFO:tensorflow:global_step/sec: 51.8205
INFO:tensorflow:probabilities = [[0.78653    0.04144535 0.04780208 ... 0.00101816 0.00032813 0.00034842]
 [0.6961342  0.04279121 0.0624396  ... 0.00090417 0.00051587 0.00061242]
 [0.85768247 0.03080463 0.02369643 ... 0.00032954 0.00047316 0.00012761]
 ...
 [0.74537516 0.02603211 0.10384403 ... 0.00027249 0.0002

INFO:tensorflow:loss = 1.0114666, step = 19401 (1.927 sec)
INFO:tensorflow:global_step/sec: 53.0396
INFO:tensorflow:probabilities = [[0.6763162  0.0604826  0.05933514 ... 0.00048109 0.00025734 0.00043815]
 [0.71201116 0.09063701 0.04685619 ... 0.00020423 0.00022412 0.00013438]
 [0.7085985  0.06933777 0.04953322 ... 0.00011181 0.00030621 0.00012464]
 ...
 [0.77566767 0.06516402 0.04217879 ... 0.00029183 0.00044638 0.00025975]
 [0.6872043  0.09327109 0.05771061 ... 0.00034118 0.0004208  0.00017481]
 [0.6523469  0.11993358 0.07334124 ... 0.00068363 0.00040669 0.00032775]] (1.886 sec)
INFO:tensorflow:loss = 2.145765, step = 19501 (1.887 sec)
INFO:tensorflow:global_step/sec: 53.1464
INFO:tensorflow:probabilities = [[0.84563315 0.02141798 0.04651025 ... 0.00002155 0.00003342 0.00001481]
 [0.86487895 0.01973423 0.04925444 ... 0.00006554 0.00008769 0.00014469]
 [0.8586516  0.03399914 0.02417247 ... 0.00010728 0.00002515 0.00003343]
 ...
 [0.79716253 0.0351251  0.06302569 ... 0.00010378 0.00010

'\nfor i in range(0,N):\n    Y[i] = AnnDict[annotations[0][i]];\n'